In [ ]:
# TVB Course Ex 3

In [1]:
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/My Drive/Colab Notebooks/TVB_Ex3')

In [4]:
os.listdir()

['Document_corpus', 'SCAIView_csv', 'Untitled0.ipynb']

In [5]:
!pip install owlready2

     |████████████████████████████████| 20.9MB 23.6MB/s 
  Created wheel for owlready2: filename=Owlready2-0.26-cp36-cp36m-linux_x86_64.whl size=20280679 sha256=ded238601e4bd1fc87a6139c6b23ef124bc8db93b2b2c9770204c8b409cff2e1
  Stored in directory: /root/.cache/pip/wheels/73/08/39/546dfe78a06872a4ae5eac478a537e6c2c601a2931d33a66a8
Successfully built owlready2


In [49]:
!pip install nipy

     |████████████████████████████████| 4.7MB 6.7MB/s 


In [51]:
!pip install nilearn

     |████████████████████████████████| 3.0MB 6.6MB/s 


In [6]:
if not os.path.isdir('Document_corpus'):
  os.mkdir('Document_corpus')
if not os.path.isdir('SCAIView_csv'):
  os.mkdir('SCAIView_csv')

In [52]:
# form notebook #Mapping Semantic Associations
import os
import numpy as np
import pandas as pd
from IPython.display import Image

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.image as mpimg

import nibabel as nib
from nibabel import freesurfer as fs
from nilearn import datasets, surface, plotting

In [7]:
# from notebook SCAIVIewAPI
import requests
import json
import csv
import os
import pandas as pd
import owlready2 as owl

In [27]:
base = 'neuro' #'academia' or 'neuro'
base_url = 'https://api.' + base + '.scaiview.com/'
def _url(path):
    return base_url + path

def search_documents(query_obj):
    '''Returns documents for a search query'''
    doc_list = requests.post(_url('api/v6/search'), json = query)
    return doc_list

def get_documents(query_obj):
    ''' Returns documents for a search query in json format'''
    doc_list = requests.post(_url('api/v6/search/documents'), json = query)
    return doc_list

# select data formats for document list: ['ID', 'CSV', 'BIBTEX', 'RIS', 'ENDNOTE']

def get_documents_list(query_obj, form='CSV', page_size='20'):
    '''Returns documents for a search query and exports to the given format.
    form describes the data format the document_list is exported to one of ['ID', 'CSV', 'BIBTEX', 'RIS', 'ENDNOTE']
    page_size: string of an integer that describes how many documents will be listed (max: '2000')
    '''
    doc_list = requests.post(_url('api/v6/search/export?format=' + form + '&page=0&size=' + page_size), json = query_obj)
    return doc_list

# terminology list (entities) SCAIView neuro

MESH = "JPMTAGGERANNOTATOR(MESH14.0-SNAPSHOT1585598140)" 
EPO = "JPMTAGGERANNOTATOR(EPO1.0-SNAPSHOT1606393218)"
UBERON = "JPMTAGGERANNOTATOR(UBERON15.0-SNAPSHOT1585594963)"
ALZHEIMER = "JPMTAGGERANNOTATOR(ALZHEIMER14.0-SNAPSHOT1585595494)"
BRCO = "JPMTAGGERANNOTATOR(BRCO14.0-SNAPSHOT1585594937)"
HOMO_SAPIENS = "JPMTAGGERANNOTATOR(HOMO_SAPIENS15.0-SNAPSHOT1586876562)"
PTS = "JPMTAGGERANNOTATOR(PTS14.0-SNAPSHOT1585595459)"
NEURONAMES = "JPMTAGGERANNOTATOR(NEURONAMES1.0-SNAPSHOT1606837450)"
FMA = "JPMTAGGERANNOTATOR(FMA1.0-SNAPSHOT1585594358)"

# terminology list (entities) SCAIView academia

UBERON = "JPMTAGGERANNOTATOR(UBERON15.0-SNAPSHOT1585594963)"
GO = "JPMTAGGERANNOTATOR(GO15.0-SNAPSHOT1585596145)"
HYPO_FINDER = "JPMTAGGERANNOTATOR(HYPOTHESISFINDER14.0-SNAPSHOT1585595494)"
DRUG_BANK = "JPMTAGGERANNOTATOR(DRUGBANK14.0-SNAPSHOT1591360597)"
HOMO_SAPIENS = "JPMTAGGERANNOTATOR(HOMO_SAPIENS15.0-SNAPSHOT1586876562)"

# Updated list # 4.2.
EPO = "JPMTAGGERANNOTATOR(EPO1.0-SNAPSHOT1608193013)"
UBERON = "JPMTAGGERANNOTATOR(UBERON15.0-SNAPSHOT1608193399)"
ALZHEIMER = "JPMTAGGERANNOTATOR(ALZHEIMER14.0-SNAPSHOT1608193157)"
BRCO = "JPMTAGGERANNOTATOR(BRCO14.0-SNAPSHOT1608194046)"
MESH = "JPMTAGGERANNOTATOR(MESH14.0-SNAPSHOT1608193912)"
PTS = "JPMTAGGERANNOTATOR(PTS14.0-SNAPSHOT1608192904)"
FMA = "JPMTAGGERANNOTATOR(FMA1.0-SNAPSHOT1608193985)"


def get_stats(query_obj, entity):
    '''Retrieves statistics about matches to the given query.'''
    corpus = requests.put(_url('api/v6/statistics/entities/' + entity), json = query)
    return corpus

def export_stats(query_obj, entity):
    '''Retrieves statistic about matches to the given query and exports the result to CSV.'''
    corpus = requests.put(_url('api/v6/statistics/entities/' + entity + '/export'), json = query_obj)
    return corpus

In [57]:
query = {"AND":{"lhs":{"AND":{"lhs":{"FREETEXT":{"searchTerm":"Parkinson"}},"rhs":{"FREETEXT":{"searchTerm":"dopamine"}}}},
                "rhs":{"TOOL":{"id":UBERON}}}}

query_name = 'dopamine'

In [67]:
query = {"AND":{"lhs":{"AND":{"lhs":{"FREETEXT":{"searchTerm":"Parkinson"}},"rhs":{"FREETEXT":{"searchTerm":"catecholamine"}}}},
                "rhs":{"TOOL":{"id":UBERON}}}}

query_name = 'catecholamine'

In [94]:
query = query = {"AND":{"lhs":{"OR":{"lhs":{"FREETEXT":{"searchTerm":"Parkinson"}},"rhs":{"FREETEXT":{"searchTerm":"Parkinson's"}}}},
                "rhs":{"CONCEPTID":{"conceptIdentifier":"mesh:D001288"}}}}

query_name = 'Attention'

In [95]:
doc_list = get_documents_list(query, 'CSV', '200')
f = open('./Document_corpus/' + query_name + '.csv', "w")
f.write(doc_list.text)
f.close()
pd.read_csv('./Document_corpus/' + query_name + '.csv').head()


,id,title,document type,publication type,references
0,PMID:7969208,Perturbation of a prehension movement in Parki...,PUBMED_ABSTRACT,"D016428:Journal Article, D013485:Research Supp...",[NOT AVAILABLE]
1,PMID:22230890,Selective attention deficits in early and mode...,PUBMED_ABSTRACT,"D016428:Journal Article, D013485:Research Supp...",[NOT AVAILABLE]
2,PMID:22759624,Relationships between cognitive functions and ...,PUBMED_ABSTRACT,"D016428:Journal Article, D016454:Review",[NOT AVAILABLE]
3,PMID:28976016,Pedestrian safety in patients with Parkinson's...,PUBMED_ABSTRACT,D016428:Journal Article,[NOT AVAILABLE]
4,PMID:18942100,A randomized controlled trial of movement stra...,PUBMED_ABSTRACT,"D003160:Comparative Study, D016428:Journal Art...",[NOT AVAILABLE]


In [96]:
corpus = export_stats(query, UBERON)

<Response [200]>
{'AND': {'lhs': {'OR': {'lhs': {'FREETEXT': {'searchTerm': 'Parkinson'}}, 'rhs': {'FREETEXT': {'searchTerm': "Parkinson's"}}}}, 'rhs': {'CONCEPTID': {'conceptIdentifier': 'mesh:D001288'}}}}


In [97]:
corpus = export_stats(query, UBERON)
f = open('./SCAIView_csv/' + query_name + '.csv', "w")
f.write(corpus.text)
f.close()
df = pd.read_csv('./SCAIView_csv/' + query_name + '.csv', names=['entity',
       'numberOfDocumentsInQuery',
       'kullbackLeiberDivergence',
       'totalNumberOfDocuments'])

<Response [200]>
{'AND': {'lhs': {'OR': {'lhs': {'FREETEXT': {'searchTerm': 'Parkinson'}}, 'rhs': {'FREETEXT': {'searchTerm': "Parkinson's"}}}}, 'rhs': {'CONCEPTID': {'conceptIdentifier': 'mesh:D001288'}}}}


In [98]:
df.head()

,entity,numberOfDocumentsInQuery,kullbackLeiberDivergence,totalNumberOfDocuments
0,UBERON:UBERON_0014899,144,0.095370,201411
1,UBERON:UBERON_0004529,118,0.224427,71750
2,UBERON:UBERON_2002175,93,0.067394,124738
3,UBERON:UBERON_0000045,85,0.310375,15950
4,UBERON:UBERON_0001851,76,0.113689,60708


In [99]:
len(df)

201

In [100]:
# import mapping between Uberon and the HCP-multimodal parcellation atlas (Glasser et al., 2016)
glasser_sheet = pd.read_csv('mapping_Glasser_Uberon.tsv',
                            sep='\t',
                            names=['Index',
                                   'AreaName',
                                   'AreaDescription',
                                   'UbMatch',
                                   'UbSuperClass',
                                   'UbSubClass',
                                   'New',
                                   'Sections',
                                   'Other Names'], skiprows=[0, 0])

glasser_cortical = glasser_sheet[0:180].drop(['New', 'Sections'], axis=1)

In [101]:
glasser_cortical.head()

,Index,AreaName,AreaDescription,UbMatch,UbSuperClass,UbSubClass,Other Names
0,1.0,V1,Primary Visual Cortex,UBERON:UBERON_0002436,"UBERON:UBERON_0000411, UBERON:UBERON_0002021, ...","UBERON:UBERON_0035906, UBERON:UBERON_0035907, ...","17, hOC1, OC, BA17"
1,2.0,MST,Medial Superior Temporal Area,NaN,"UBERON:UBERON_0016538, UBERON:UBERON_0001871, ...",NaN,"MSTv, hOC5, hOC5v"
2,3.0,V6,Sixth Visual Area,NaN,"UBERON:UBERON_0000411, UBERON:UBERON_0002021, ...",NaN,112
3,4.0,V2,Second Visual Area,UBERON:UBERON_0022232,"UBERON:UBERON_0000411, UBERON:UBERON_0002021, ...",NaN,"18, hOC2, OB, BA18"
4,5.0,V3,Third Visual Area,NaN,"UBERON:UBERON_0000411, UBERON:UBERON_0002021, ...",NaN,"V3d, V3v, VP, hOC3d, hOC3v"


In [102]:
wd = './SCAIView_csv/' # Directory of SCAIView output.

df_scai = pd.DataFrame() # Create empty data frame to store transformed data.
df_scai['Area'] = glasser_cortical['AreaName'] # Attach Glasser areas.

# Loop through output files.
for file in os.listdir(wd):

    if file.endswith('.csv'):
        print(file)
    else:
        continue # Skip files with wrong format.
    
    # Read out SCAIView output.
    scai_csv = pd.read_csv(wd + file,
                           names=['ID',
                                  'Count of Documents',
                                  'Relevance',
                                  'Total documents'])

    f = file.replace(' ', '_').replace('.csv', '') # Filename.

    ID_list = scai_csv['ID'].values

    # Create empty value vectors.
    rel_sub = [0] * 180 #for subclasses
    rel_super = [0] * 180 #for super-classes
    rel_match = [0] * 180 #for exact match

    # Support vector, counting super- and subclasses to avergage if >1.
    v_super = [1] * 180
    v_sub = [1] * 180

    # Parse through Uberon-IDs and compare with Uberon-Glasser mapping.
    for index, row in scai_csv.iterrows():
        ID = row['ID']
        value = row['Relevance']

        for i, area in glasser_cortical.iterrows():
            
            match = str(area['UbMatch'])
            UbSuperClass = str(area['UbSuperClass']).replace(',', '').split()
            UbSubClass = str(area['UbSubClass']).replace(',', '').split()
            
            
            # Coefficient for >1 SuperClasses.
            t = 0
            for c in UbSuperClass:
                if c in ID_list:
                    t += 1
            if t > 0:
                v_super[i] = t
            else:
                v_super[i] = 1

                
            # Coefficient for >1 SubClasses.
            t = 0
            for c in UbSubClass:
                if c in ID_list:
                    t += 1
            if t > 0:
                v_sub[i] = t
            else:
                v_sub[i] = 1
            
            
            # Compare ID with area and assign relevance score.
            if ID in match:
                rel_match[i] += value

            elif ID in UbSubClass:
                rel_sub[i] += value

            elif ID in UbSuperClass:
                rel_super[i] += value

            else:
                pass

            
    # Average relevance scores over super- and subclasses.
    
    # SubClasses.
    rel_sub_av = [a / b for a, b in zip(rel_sub, v_sub)]

    # SuperClasses.
    rel_sup_av = [c / d for c, d in zip(rel_super, v_super)]

    # Sum score.
    rel_av = [i + j + k for i, j, k in zip(rel_sub_av, rel_sup_av, rel_match)]
    
    
    df_scai[f] = rel_av # Add to data frame.
    

catecholamines.csv
dopamine.csv
catecholamine.csv
Attention.csv


In [103]:
df_scai.head()

,Area,catecholamines,dopamine,catecholamine,Attention
0,V1,0.0,0.009570,0.077087,0.011020
1,MST,0.0,-0.001390,0.000000,0.001467
2,V6,0.0,0.009570,0.077087,0.011020
3,V2,0.0,0.006573,0.077087,0.011020
4,V3,0.0,0.009570,0.077087,0.011020


In [104]:
# %% Define Colormap
# Remove the top 10% and reverse colormap
interval = np.hstack(np.linspace(0, 0.9))
colors = plt.cm.hot_r(interval)
cmap = LinearSegmentedColormap.from_list('name', colors)

In [105]:
# Choose template brain surface.
subj_dir = "./subjects"  # Locally under $FREESURFER_HOME/subjects
subject_id = "fsaverage" 
hemi = "rh"
surf = ".inflated"

# Load surface.
surf_file = os.path.join(subj_dir,subject_id,
                         "surf",
                         hemi + surf)

surf = nib.freesurfer.io.read_geometry(surf_file)


# Load annotation.
aparc_file = os.path.join(subj_dir,
                          subject_id, "label",
                          hemi + ".HCP-MMP1.annot") # Define directory of parcellation file.

labels, ctab, names = nib.freesurfer.read_annot(aparc_file) # Readout parcellation as freesurfer .annot file.

In [106]:
df_scai_norm = pd.DataFrame() # Empty data frame for normalized scores.

In [110]:
for column in df_scai.iloc[:, 1:]:
    m = df_scai[column]
    
    # normalize (rescale to range between 0 and 1)
    if np.min(m) == np.max(m): # Account for cases with max = min
        m[m == np.max(m)] = 1
    else:
        m = (m - np.min(m))/(np.max(m)-np.min(m))

    # transform vector
    m = list(m)
    m = [float(i) for i in m]
    m = np.array(m)
    m = np.nan_to_num(m)
    
    # insert 0 for unknown/subcortical areas
    m = np.insert(m, 0, 0)
    
    # save to new dataframe
    df_scai_norm[column] = m

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [111]:
df_scai_norm.tail()

,catecholamines,dopamine,catecholamine,Attention
176,1.0,0.092230,0.0,0.080662
177,1.0,0.053963,0.0,0.080662
178,1.0,0.107081,0.0,0.067030
179,1.0,0.286185,0.0,0.495437
180,1.0,0.286185,0.0,0.495437


In [112]:
# Choose map of interest.
moi = 'Attention' # !Insert MeSH query from the group!
m = df_scai_norm[moi]
# Upsample vertex-data: align 
vtx_data = m[labels]
vtx_data = np.array(vtx_data)
view = plotting.view_surf(surf, vtx_data,
                          cmap=cmap,
                          symmetric_cmap=False, 
                          black_bg=True)
view

Output hidden; open in https://colab.research.google.com to view.

In [117]:
vtx_data.shape

(163842,)

In [86]:
# Choose map of interest.
moi = 'catecholamine' # !Insert MeSH query from the group!
m = df_scai_norm[moi]

In [87]:
# Upsample vertex-data: align 
vtx_data = m[labels]

In [88]:
vtx_data = np.array(vtx_data)
view = plotting.view_surf(surf, vtx_data,
                          cmap=cmap,
                          symmetric_cmap=False, 
                          black_bg=True)

In [89]:
view

Output hidden; open in https://colab.research.google.com to view.

In [93]:
# Choose map of interest.
moi = 'dopamine' # !Insert MeSH query from the group!
m = df_scai_norm[moi]
# Upsample vertex-data: align 
vtx_data = m[labels]
vtx_data = np.array(vtx_data)
view = plotting.view_surf(surf, vtx_data,
                          cmap=cmap,
                          symmetric_cmap=False, 
                          black_bg=True)
view

Output hidden; open in https://colab.research.google.com to view.